In [1]:
import pandas as pd
import numpy as np

In [3]:
file_train=r'/Users/Dell/Documents/loan_data_train.csv'
file_test=r'/Users/Dell/Documents/loan_data_test.csv'



In [4]:
ld_train=pd.read_csv(file_train)
ld_test=pd.read_csv(file_test)

In [5]:
ld_train.head()
len(ld_train)

2200

In [6]:
ld_train['data']='train'
ld_test['data']='test'

In [7]:
ld_test['Interest.Rate']=np.nan
ld_test.head()
ld_test.shape
ld_train.shape

(2200, 16)

In [8]:
ld_all=pd.concat([ld_train,ld_test],axis=0)


In [9]:
ld_all.shape

(2500, 16)

In [10]:
ld_all.dtypes

Amount.Funded.By.Investors         object
Amount.Requested                   object
Debt.To.Income.Ratio               object
Employment.Length                  object
FICO.Range                         object
Home.Ownership                     object
ID                                float64
Inquiries.in.the.Last.6.Months    float64
Interest.Rate                      object
Loan.Length                        object
Loan.Purpose                       object
Monthly.Income                    float64
Open.CREDIT.Lines                  object
Revolving.CREDIT.Balance           object
State                              object
data                               object
dtype: object

In [11]:
ld_all.drop(['ID','Amount.Funded.By.Investors'],axis=1,inplace=True)

In [12]:
ld_all.select_dtypes(['object']).columns

Index(['Amount.Requested', 'Debt.To.Income.Ratio', 'Employment.Length',
       'FICO.Range', 'Home.Ownership', 'Interest.Rate', 'Loan.Length',
       'Loan.Purpose', 'Open.CREDIT.Lines', 'Revolving.CREDIT.Balance',
       'State', 'data'],
      dtype='object')

In [13]:
for col in ['Debt.To.Income.Ratio','Interest.Rate']:
    ld_all[col]=ld_all[col].str.replace('%','')

In [14]:
for col in ['Debt.To.Income.Ratio','Amount.Requested','Interest.Rate','Open.CREDIT.Lines','Revolving.CREDIT.Balance']:
    ld_all[col]=pd.to_numeric(ld_all[col],errors='coerce')

In [15]:
ld_all['Employment.Length'].unique()

array(['5 years', '4 years', '< 1 year', '10+ years', nan, '2 years',
       '8 years', '1 year', '6 years', '7 years', '3 years', '9 years',
       '.'], dtype=object)

In [16]:
ld_all['Employment.Length']=ld_all['Employment.Length'].str.replace('years','')
ld_all['Employment.Length']=ld_all['Employment.Length'].str.replace('year','')
ld_all['Employment.Length']=np.where(ld_all['Employment.Length'].str[:2]=='10',10,ld_all['Employment.Length'])
ld_all['Employment.Length']=np.where(ld_all['Employment.Length'].str[0]=='<','',ld_all['Employment.Length'])
ld_all['Employment.Length']=pd.to_numeric(ld_all['Employment.Length'],errors='coerce')

In [17]:
ld_all['Employment.Length'].dtypes

dtype('float64')

In [18]:
ld_all.dtypes

Amount.Requested                  float64
Debt.To.Income.Ratio              float64
Employment.Length                 float64
FICO.Range                         object
Home.Ownership                     object
Inquiries.in.the.Last.6.Months    float64
Interest.Rate                     float64
Loan.Length                        object
Loan.Purpose                       object
Monthly.Income                    float64
Open.CREDIT.Lines                 float64
Revolving.CREDIT.Balance          float64
State                              object
data                               object
dtype: object

In [19]:
ld_all['FICO.Range'].unique()

array(['720-724', '710-714', '690-694', '730-734', '785-789', '665-669',
       '735-739', '740-744', '765-769', '800-804', '680-684', '745-749',
       '660-664', '670-674', '705-709', '695-699', '715-719', '760-764',
       '685-689', '755-759', '675-679', '775-779', '725-729', '640-644',
       '790-794', '770-774', '700-704', '750-754', '805-809', '810-814',
       '815-819', '780-784', '820-824', '795-799', '645-649', '655-659',
       '830-834', '650-654'], dtype=object)

In [20]:
k=ld_all['FICO.Range'].str.split('-',expand=True).astype(float)


In [21]:
ld_all['fico']=0.5*(k[0]+k[1])

In [22]:
del ld_all['FICO.Range']

In [23]:
cat_cols=ld_all.select_dtypes(['object']).columns
cat_cols

Index(['Home.Ownership', 'Loan.Length', 'Loan.Purpose', 'State', 'data'], dtype='object')

In [24]:
cat_cols=cat_cols[:-1]
cat_cols

Index(['Home.Ownership', 'Loan.Length', 'Loan.Purpose', 'State'], dtype='object')

In [25]:
for col in cat_cols:
    freqs=ld_all[col].value_counts()
    k=freqs.index[freqs>20][:-1]
    for cat in k:
        name=col+'_'+cat
        ld_all[name]=(ld_all[col]==cat).astype(int)
        
    del ld_all[col]
    print(col)
    
    

Home.Ownership
Loan.Length
Loan.Purpose
State


In [26]:
ld_all.head()

,Amount.Requested,Debt.To.Income.Ratio,Employment.Length,Inquiries.in.the.Last.6.Months,Interest.Rate,Monthly.Income,Open.CREDIT.Lines,Revolving.CREDIT.Balance,data,fico,...,State_MN,State_AL,State_MO,State_NV,State_OR,State_SC,State_WI,State_KY,State_LA,State_OK
0,25000.0,27.56,5.0,3.0,18.49,8606.56,11.0,15210.0,train,722.0,...,0,0,0,0,0,0,0,0,0,0
1,19750.0,13.39,4.0,3.0,17.27,6737.50,14.0,19070.0,train,712.0,...,0,0,0,0,0,0,0,0,0,0
2,2100.0,3.50,NaN,1.0,14.33,1000.00,13.0,893.0,train,692.0,...,0,0,0,0,0,0,0,0,1,0
3,28000.0,19.62,10.0,1.0,16.29,7083.33,12.0,38194.0,train,712.0,...,0,0,0,1,0,0,0,0,0,0
4,24250.0,23.79,10.0,2.0,12.23,5833.33,6.0,31061.0,train,732.0,...,0,0,0,0,0,0,0,0,0,0


In [27]:
ld_all.dtypes

Amount.Requested                   float64
Debt.To.Income.Ratio               float64
Employment.Length                  float64
Inquiries.in.the.Last.6.Months     float64
Interest.Rate                      float64
Monthly.Income                     float64
Open.CREDIT.Lines                  float64
Revolving.CREDIT.Balance           float64
data                                object
fico                               float64
Home.Ownership_MORTGAGE              int32
Home.Ownership_RENT                  int32
Loan.Length_36 months                int32
Loan.Purpose_debt_consolidation      int32
Loan.Purpose_credit_card             int32
Loan.Purpose_other                   int32
Loan.Purpose_home_improvement        int32
Loan.Purpose_major_purchase          int32
Loan.Purpose_small_business          int32
Loan.Purpose_car                     int32
Loan.Purpose_wedding                 int32
Loan.Purpose_medical                 int32
Loan.Purpose_moving                  int32
State_CA   

In [28]:
ld_all.isnull().sum()

Amount.Requested                     5
Debt.To.Income.Ratio                 1
Employment.Length                  329
Inquiries.in.the.Last.6.Months       3
Interest.Rate                      300
Monthly.Income                       3
Open.CREDIT.Lines                    9
Revolving.CREDIT.Balance             5
data                                 0
fico                                 0
Home.Ownership_MORTGAGE              0
Home.Ownership_RENT                  0
Loan.Length_36 months                0
Loan.Purpose_debt_consolidation      0
Loan.Purpose_credit_card             0
Loan.Purpose_other                   0
Loan.Purpose_home_improvement        0
Loan.Purpose_major_purchase          0
Loan.Purpose_small_business          0
Loan.Purpose_car                     0
Loan.Purpose_wedding                 0
Loan.Purpose_medical                 0
Loan.Purpose_moving                  0
State_CA                             0
State_NY                             0
State_TX                 

In [29]:
for col in ld_all.columns:
    if (ld_all[col].isnull().sum()>0) & (col not in ['Interest.Rate','data']):
        ld_all.loc[ld_all[col].isnull(),col]=ld_all.loc[ld_all['data']=='train',col].mean()

In [30]:
ld_train=ld_all[ld_all['data']=='train']
ld_test=ld_all[ld_all['data']=='test']

In [31]:
del ld_train['data']


In [32]:
ld_test.drop(['data','Interest.Rate'],axis=1,inplace=True)

C:\Users\Dell\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [34]:
ld_train.shape

(2200, 50)

In [35]:
from sklearn.model_selection import train_test_split, GridSearchCV

In [36]:
ld_train1,ld_train2=train_test_split(ld_train,test_size=0.2,random_state=2)

In [37]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error

In [46]:
lm=LinearRegression()

In [50]:
x_train=ld_train1.drop(['Interest.Rate'],axis=1)
y_train=ld_train1['Interest.Rate']

In [48]:
x_test=ld_train2.drop(['Interest.Rate'],axis=1)
y_test=ld_train2['Interest.Rate']

In [51]:
lm.fit(x_train,y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

In [52]:
predict_val=lm.predict(x_test)

In [53]:
residual=y_test-predict_val

In [55]:
rmse=np.sqrt(np.dot(residual,residual)/len(predict_val))
rmse

2.0359227285440467

In [56]:
x_train=ld_train.drop(['Interest.Rate'],axis=1)
y_train=ld_train['Interest.Rate']

In [57]:
lm.fit(x_train,y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

In [60]:
predict_ir=lm.predict(ld_test)

# RIDGE REGRESSION

In [65]:
from sklearn.linear_model import Ridge,Lasso
lambdas=np.linspace(1,100,100)

In [66]:
params={'alpha':lambdas}

In [63]:
x_train.reset_index(inplace=True,drop=True)
y_train.reset_index(inplace=True,drop=True)

In [67]:
ridge=Ridge(fit_intercept=True)

In [68]:
grid_search=GridSearchCV(ridge,param_grid=params,cv=10,scoring='neg_mean_absolute_error',n_jobs=-1,verbose=20)


In [69]:
grid_search

GridSearchCV(cv=10, error_score='raise',
       estimator=Ridge(alpha=1.0, copy_X=True, fit_intercept=True, max_iter=None,
   normalize=False, random_state=None, solver='auto', tol=0.001),
       fit_params=None, iid=True, n_jobs=-1,
       param_grid={'alpha': array([  1.,   2., ...,  99., 100.])},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='neg_mean_absolute_error', verbose=20)

In [70]:
grid_search.fit(x_train,y_train)

Fitting 10 folds for each of 100 candidates, totalling 1000 fits


[Parallel(n_jobs=-1)]: Done   1 tasks      | elapsed:    1.7s
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:    2.5s
[Parallel(n_jobs=-1)]: Done   3 tasks      | elapsed:    3.8s
[Parallel(n_jobs=-1)]: Done   4 tasks      | elapsed:    4.7s
[Parallel(n_jobs=-1)]: Done   5 tasks      | elapsed:    4.7s
[Parallel(n_jobs=-1)]: Done   6 tasks      | elapsed:    4.7s
[Parallel(n_jobs=-1)]: Done   7 tasks      | elapsed:    4.7s
[Parallel(n_jobs=-1)]: Done   8 tasks      | elapsed:    4.7s
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:    4.7s
[Parallel(n_jobs=-1)]: Done  10 tasks      | elapsed:    4.7s
[Parallel(n_jobs=-1)]: Done  11 tasks      | elapsed:    4.7s
[Parallel(n_jobs=-1)]: Done  12 tasks      | elapsed:    4.7s
[Parallel(n_jobs=-1)]: Done  13 tasks      | elapsed:    4.7s
[Parallel(n_jobs=-1)]: Done  14 tasks      | elapsed:    4.8s
[Parallel(n_jobs=-1)]: Done  15 tasks      | elapsed:    4.8s
[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed:    4.8s
[Paralle

GridSearchCV(cv=10, error_score='raise',
       estimator=Ridge(alpha=1.0, copy_X=True, fit_intercept=True, max_iter=None,
   normalize=False, random_state=None, solver='auto', tol=0.001),
       fit_params=None, iid=True, n_jobs=-1,
       param_grid={'alpha': array([  1.,   2., ...,  99., 100.])},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='neg_mean_absolute_error', verbose=20)

In [71]:
grid_search.best_estimator_

Ridge(alpha=32.0, copy_X=True, fit_intercept=True, max_iter=None,
   normalize=False, random_state=None, solver='auto', tol=0.001)

In [72]:
grid_search.cv_results_

C:\Users\Dell\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:122: FutureWarning: You are accessing a training score ('mean_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
C:\Users\Dell\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:122: FutureWarning: You are accessing a training score ('split0_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
C:\Users\Dell\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:122: FutureWarning: You are accessing a training score ('split1_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
C:\Users\Dell\Anaconda3\lib\site-packages\sklearn\utils\depr

{'mean_fit_time': array([0.05011017, 0.01562479, 0.02187629, 0.01718731, 0.03072441,
        0.01840014, 0.01840262, 0.01800206, 0.01440058, 0.01024449,
        0.00999091, 0.00668793, 0.00156226, 0.01250014, 0.01562531,
        0.00781231, 0.00937488, 0.01406288, 0.01406283, 0.02031293,
        0.01093771, 0.01250005, 0.00781255, 0.00312514, 0.00468726,
        0.01406243, 0.01406245, 0.01093779, 0.01406264, 0.01406195,
        0.01093752, 0.00937657, 0.014062  , 0.01093762, 0.01562507,
        0.0156255 , 0.01718843, 0.01718864, 0.01562512, 0.01406279,
        0.01093771, 0.01406269, 0.01406262, 0.01093748, 0.01562479,
        0.0125005 , 0.01875031, 0.00468814, 0.00937521, 0.01093733,
        0.01249993, 0.01562505, 0.01250038, 0.00625   , 0.00937517,
        0.01406267, 0.00937505, 0.00937507, 0.01718764, 0.00937502,
        0.00781236, 0.01562548, 0.01875036, 0.01562631, 0.01093888,
        0.00468743, 0.00937471, 0.00781267, 0.00156305, 0.01718843,
        0.01406212, 0.01570024,

In [ ]:
x_train2=ld_train2.drop(['Interest.Rate'],axis=1)
y_train2=ld_train2['Interest.Rate']

In [ ]:
lm.fit(x_train1,y_train1)

In [ ]:
predict_val=lm.predict(x_train2)

In [ ]:
mae=mean_absolute_error(y_train2,predict_val)
mae

In [ ]:
error=y_train2-predict_val

In [ ]:
rmse=np.sqrt(np.dot(error,error)/len(predict_val))
rmse

In [ ]:
x_train=ld_train.drop(['Interest.Rate'],axis=1)
y_train=ld_train['Interest.Rate']

In [ ]:
lm.fit(x_train,y_train)

In [ ]:
predict_ir=lm.predict(ld_test)

# Ridge Regression

In [ ]:
lambdas=np.linspace(1,100,100)

In [ ]:
ld_train.reset_index(inplace=True,drop=True)

In [ ]:
x_train=ld_train.drop(['Interest.Rate'],axis=1)
y_train=ld_train['Interest.Rate']

In [ ]:
mae_list=[]
for a in lambdas:
    ridge=Ridge(fit_intercept=True,alpha=a)
    kf=KFold(n_splits=10)
    xval_err=0
    for train,test in kf.split(x_train):
        ridge.fit(x_train.loc[train],y_train)
        p=ridge.predict(x_train.loc[test])
        xval_err+=mean_absolute_error(y_train[test],p)
    
    mae10_cv=xval_err/10
    print(a,':',mae10_cv)
    mae_list.extend([mae10_cv])
        

In [ ]:
best_alppha=lambdas[mae_list==min(mae_list)]
print('Aplha with min 10cv error is',best_alpha)

In [ ]:
from sklearn.linear_model import Ridge,Lasso
from sklearn.model_selection import KFold

In [ ]:
lambdas=np.linspace(1,100,100)

In [ ]:
ld_train.reset_index(inplace=True,drop=True)

In [ ]:
x_train=ld_train.drop(['Interest.Rate'],axis=1)
y_train=ld_train['Interest.Rate']

In [ ]:
mae_list=[]
for a in lambdas:
    ridge=Ridge(fit_intercept=True,alpha=a)
    kf=KFold(n_splits=10)
    xval_err=0
    
    for train,test in kf.split(x_train):
        ridge.fit(x_train.loc[train],y_train[train])
        p=ridge.predict(x_train.loc[test])
        xval_err+=mean_absolute_error(y_train[test],p)
    
    mae_10cv=xval_err/10
    print(a,':',mae_10cv)
    mae_list.extend([mae_10cv])
    
      

In [ ]:
best_alpha=lambdas[mae_list==min(mae_list)]
print('value of Alpha with min mean absolute error',min(mae_list),best_alpha)

In [ ]:
help(ridge.fit(x_train,y_train))

In [ ]:
ridge.fit(x_train,y_train)

In [ ]:
test_pred=ridge.predict(ld_test)

ridge.intercept_


# Lasso Regression

In [ ]:
lambdas=np.linspace(0.001,2,100)

In [ ]:
ld_train.reset_index(inplace=True,drop=True)

In [ ]:
x_train=ld_train.drop(['Interest.Rate'],axis=1)
y_train=ld_train['Interest.Rate']

In [ ]:
mae_list=[]
for a in lambdas:
    lasso=Lasso(fit_intercept=True,alpha=a)
    kf=KFold(n_splits=10)
    xval_err=0
    for train,test in kf.split(x_train):
        lasso.fit(x_train.loc[train],y_train[train])
        p=lasso.predict(y_train[test])
        xval_err+=mean_absolute_error(y_train[test],p)
    
    mae_10cv=xval_err/10
    print(a,':',mae_10cv)
    mae_list.extend([mae_10cv])
        

In [ ]:
best_alpha=lambdas[mae_list=min(mae_list)]
print('Apha with min 10 cv error is',best_alpha)

In [ ]:
lasso=Lasso(fit_intercept=True,a=best_alpha)
lasso.fit(x_train,y_train)


In [ ]:
mae_list=[]

for a in lambdas:
    lasso=Lasso(fit_intercept=True,alpha=a)
    kf=KFold(n_splits=10)
    xval_err=0
    for train,test in kf.split(x_train):
        lasso.fit(x_train.loc[train],y_train[train])
        p=lasso.predict(x_train.loc[test])
        xval_err+=mean_absolute_error(y_train[test],p)
        
    mae_10cv=xval_err/10
    print(a,':',mae_10cv)
    mae_list.extend([mae_10cv])

best_alpha=lambdas[mae_list==min(mae_list)]
print('Alpha with min 10cv error',best_alpha)


    
    
        